# 🐄 Sistema de Estimación de Peso Bovino - Setup ML

> **BLOQUE 0**: Información del proyecto (markdown - solo lectura)

**Proyecto**: Hacienda Gamelera - Bruno Brito Macedo  
**Responsable**: Persona 2 - Setup Infraestructura ML  
**Objetivo**: Preparar datasets y pipeline para entrenamiento de 7 modelos por raza  
**Duración**: 5-6 días  

---

## 📑 Índice de Bloques (Referencia Rápida)

| Bloque | Nombre | Descripción | Requisitos |
|--------|--------|-------------|------------|
| **0** | Información | Markdown introductorio | Ninguno |
| **1** | Clonar Repositorio | Monta Drive y clona desde GitHub (persistente) | Ninguno (requiere internet) |
| **2** | Importar Módulos | Importa módulos internos | Bloque 1 |
| **3** | Ejemplo Modelo | Bloque de prueba (opcional) | Bloque 2 |
| **4** | Instalar Dependencias Críticas | TensorFlow 2.17, NumPy 1.26.4, MLflow, ml_dtypes | Ninguno |
| **5** | Instalar Complementos | Albumentations, OpenCV, herramientas ML y verificaciones | Bloque 4 |
| **6** | Imports Generales | Pandas, numpy, tensorflow | Bloque 5 |
| **7** | Configuración Proyecto | Crea carpetas en Drive (Drive ya montado en Bloque 1) | Bloque 6 |
| **7.5** | Configurar Variables CID | Configura rutas CID Dataset | Bloque 7 (opcional) |
| **8** | CID Dataset | Extrae CID Dataset | Bloque 7.5 + archivo comprimido |
| **8.5** | Configurar Kaggle.json | Copia kaggle.json desde Drive | Bloque 7 (opcional) |
| **9** | Kaggle Dataset | Descarga dataset Kaggle | Bloque 8.5 + kaggle.json |
| **10** | Google Images | Scraping opcional | Bloque 7 (opcional) |
| **11** | Resumen Datasets | Muestra resumen | Bloques 8-10 |
| **12** | EDA CID Dataset | Análisis exploratorio | Bloque 8 + metadata.csv |
| **13** | Visualizaciones EDA | Gráficos interactivos | Bloque 12 |
| **14** | Análisis por Raza | Análisis por raza | Bloque 12 |
| **15** | Pipeline de Datos | Pipeline con augmentation | Bloque 12 |
| **16** | Arquitectura Modelo | Crea modelo EfficientNetB0 | Bloque 15 |
| **17** | Configurar Entrenamiento | Callbacks y MLflow | Bloque 16 |
| **18** | Entrenamiento | Entrena modelo (2-4h) | Bloque 17 + GPU |
| **19** | Evaluación | Evalúa modelo (R², MAE) | Bloque 18 |
| **20** | Exportar TFLite | Exporta a TFLite | Bloque 19 |
| **21** | Resumen Final | Genera resumen completo | Todos los bloques |

---

## 📋 Checklist de Tareas
- [x] Día 1: Setup Google Colab Pro + dependencias
- [ ] Día 2-3: Descargar y organizar datasets críticos
- [ ] Día 4: Análisis exploratorio de datos (EDA)
- [ ] Día 5-6: Preparar pipeline de datos optimizado

## 🎯 Razas Objetivo (7 razas)
1. **Brahman** - Bos indicus robusto
2. **Nelore** - Bos indicus
3. **Angus** - Bos taurus, buena carne
4. **Cebuinas** - Bos indicus general
5. **Criollo** - Adaptado local
6. **Pardo Suizo** - Bos taurus grande
7. **Jersey** - Lechera, menor tamaño


In [ ]:
# ============================================================
# BLOQUE 1: CONFIGURAR RUTA DEL PROYECTO Y CLONAR REPOSITORIO EN DRIVE
# ============================================================
# 📁 Clona el repositorio desde GitHub a Google Drive (persistente entre sesiones)
# 🔗 Repositorio: https://github.com/Angello-27/bovine-weight-estimation.git
# 💾 Se clona en Drive para que persista entre desconexiones del runtime

import sys
import subprocess
import os
from pathlib import Path

# 🔗 URL del repositorio de GitHub
GITHUB_REPO_URL = 'https://github.com/Angello-27/bovine-weight-estimation.git'

# 🔑 Montar Google Drive primero (si no está montado)
print("🔗 Montando Google Drive...")
try:
    from google.colab import drive
    
    # Verificar si Drive ya está montado
    drive_path = Path('/content/drive')
    if not drive_path.exists() or not any(drive_path.iterdir()):
        print("📁 Google Drive no está montado. Montando ahora...")
        print("💡 Se solicitará autorización para acceder a Google Drive")
        drive.mount('/content/drive')
        print("✅ Google Drive montado exitosamente")
    else:
        print("✅ Google Drive ya está montado")
    
    # Usar Drive como ubicación predeterminada (persistente)
    DRIVE_BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')
    USE_DRIVE = True
    
except ImportError:
    print("⚠️ No se puede montar Google Drive (no estamos en Colab)")
    print("💡 Usando /content/ como ubicación temporal")
    DRIVE_BASE_DIR = None
    USE_DRIVE = False
except Exception as e:
    print(f"⚠️ Error al montar Google Drive: {e}")
    print("💡 Usando /content/ como ubicación temporal")
    DRIVE_BASE_DIR = None
    USE_DRIVE = False

# ✅ Ruta donde se clonará o está el repositorio
#    - Preferencia 1: Google Drive (persistente entre sesiones)
#    - Preferencia 2: /content/ (temporal, se pierde al desconectar)
if USE_DRIVE and DRIVE_BASE_DIR:
    BASE_DIR = DRIVE_BASE_DIR
    print(f"\n💾 Usando Google Drive: {BASE_DIR} (persistente entre sesiones)")
else:
    BASE_DIR = Path('/content/bovine-weight-estimation')
    print(f"\n⚠️ Usando ubicación temporal: {BASE_DIR} (se pierde al desconectar)")
    print("💡 Para persistir, monta Google Drive primero")

# Ruta del proyecto ML Training
ML_TRAINING_DIR = BASE_DIR / 'ml-training'

# Validamos que la estructura del proyecto exista antes de continuar.
if ML_TRAINING_DIR.exists() and (ML_TRAINING_DIR / 'src').exists():
    print(f"\n✅ Proyecto ya existe en: {ML_TRAINING_DIR}")
    print("📂 Subcarpetas clave detectadas:")
    print(f"   - Código fuente: {ML_TRAINING_DIR / 'src'}")
    print(f"   - Scripts utilitarios: {ML_TRAINING_DIR / 'scripts'}")
    print(f"   - Configuración: {ML_TRAINING_DIR / 'config'}")
    if USE_DRIVE:
        print(f"\n💾 El proyecto persiste entre sesiones porque está en Google Drive")
else:
    print(f"\n📥 Proyecto no encontrado en: {ML_TRAINING_DIR}")
    print(f"🔗 Clonando repositorio desde GitHub: {GITHUB_REPO_URL}")
    if USE_DRIVE:
        print(f"💾 Se clonará en Google Drive para que persista entre sesiones")
    else:
        print(f"⚠️ Se clonará en /content/ (temporal, se perderá al desconectar)")
    
    # Clonar repositorio si no existe
    try:
        # Crear directorio padre si no existe
        BASE_DIR.parent.mkdir(parents=True, exist_ok=True)
        
        # Eliminar directorio si existe pero está vacío o incompleto
        if BASE_DIR.exists() and not (ML_TRAINING_DIR / 'src').exists():
            print(f"⚠️ Directorio existe pero incompleto. Eliminando {BASE_DIR}...")
            import shutil
            shutil.rmtree(BASE_DIR, ignore_errors=True)
        
        # Clonar repositorio
        print(f"📥 Clonando repositorio...")
        result = subprocess.run(
            ['git', 'clone', GITHUB_REPO_URL, str(BASE_DIR)],
            capture_output=True,
            text=True,
            check=True
        )
        
        print(f"✅ Repositorio clonado exitosamente en: {BASE_DIR}")
        print(f"📂 Estructura del proyecto:")
        print(f"   - Código fuente: {ML_TRAINING_DIR / 'src'}")
        print(f"   - Scripts utilitarios: {ML_TRAINING_DIR / 'scripts'}")
        print(f"   - Configuración: {ML_TRAINING_DIR / 'config'}")
        
        if USE_DRIVE:
            print(f"\n💾 El proyecto está en Google Drive y persistirá entre sesiones")
            print(f"💡 No necesitarás volver a clonarlo en futuras sesiones")
        else:
            print(f"\n⚠️ El proyecto está en /content/ y se perderá al desconectar el runtime")
            print(f"💡 Considera montar Google Drive y volver a ejecutar este bloque")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Error al clonar repositorio: {e}")
        print(f"   stdout: {e.stdout}")
        print(f"   stderr: {e.stderr}")
        print("\n💡 Soluciones:")
        print("   1. Verifica que tienes conexión a internet en Colab")
        print("   2. Verifica que el repositorio existe: https://github.com/Angello-27/bovine-weight-estimation")
        print("   3. Si el repositorio es privado, ejecuta el BLOQUE 0.5 primero para configurar el token")
        raise
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        raise

# Añadimos la carpeta src al PYTHONPATH para que todos los módulos internos sean importables.
sys.path.insert(0, str(ML_TRAINING_DIR / 'src'))

# Verificación final
if ML_TRAINING_DIR.exists() and (ML_TRAINING_DIR / 'src').exists():
    print(f"\n✅ Configuración completada correctamente")
    print(f"📁 Directorio base: {BASE_DIR}")
    print(f"📁 ML Training: {ML_TRAINING_DIR}")
    print(f"🐍 PYTHONPATH actualizado: {ML_TRAINING_DIR / 'src'}")
    if USE_DRIVE:
        print(f"💾 Ubicación: Google Drive (persistente entre sesiones)")
    else:
        print(f"⚠️ Ubicación: /content/ (temporal, se pierde al desconectar)")
else:
    raise RuntimeError(
        f"No se pudo configurar el proyecto. Verifica que {ML_TRAINING_DIR} existe y contiene 'src'."
    )


In [ ]:
# ============================================================
# BLOQUE 2: IMPORTAR MÓDULOS DEL PROYECTO
# ============================================================
# ✅ Importa módulos internos del proyecto (requiere BLOQUE 1 exitoso)

# Data Augmentation
from data.augmentation import get_training_transform, get_aggressive_augmentation, get_validation_transform

# Modelos
from models.cnn_architecture import BreedWeightEstimatorCNN, BREED_CONFIGS

# Evaluación
from models.evaluation.metrics import MetricsCalculator, ModelMetrics

# Exportación TFLite
from models.export.tflite_converter import TFLiteExporter

print("✅ Todos los módulos importados correctamente")
print("\n📦 Módulos disponibles:")
print("   - Data augmentation (Albumentations 2.0.8)")
print("   - CNN architectures (MobileNetV2, EfficientNet)")
print("   - Metrics calculator (R², MAE, MAPE)")
print("   - TFLite exporter (optimizado para móvil)")


In [ ]:
# ============================================================
# BLOQUE 3: EJEMPLO - CREAR MODELO PARA UNA RAZA (OPCIONAL)
# ============================================================
# 🎓 Bloque de prueba para verificar que los módulos funcionan
# ⚠️ Puedes omitir este bloque si ya sabes que todo funciona

# Ejemplo 1: Crear modelo para Brahman
model_brahman = BreedWeightEstimatorCNN.build_model(
    breed_name='brahman',
    base_architecture='mobilenetv2'  # Más rápido que EfficientNet
)

print(f"✅ Modelo creado: {model_brahman.name}")
print(f"📊 Parámetros: {model_brahman.count_params():,}")

# Ver arquitectura
print("\n📐 Arquitectura del modelo:")
model_brahman.summary()


---

## 📝 Próximos Pasos

1. **Descargar datasets** (CID, CattleEyeView, etc.)
2. **Preprocesar datos** con nuestros módulos
3. **Entrenar modelo base** genérico
4. **Fine-tuning por raza** (5 razas)
5. **Recolección propia** (Criollo, Pardo Suizo)
6. **Exportar a TFLite** e integrar en app móvil

> Ver `README.md` y `scripts/train_all_breeds.py` para más ejemplos.



## 🚀 Día 1: Setup Google Colab Pro + Dependencias

In [ ]:
# ============================================================
# BLOQUE 3.5: LIMPIEZA DE DEPENDENCIAS CONFLICTIVAS
# ============================================================
# 🧹 Limpia versiones antiguas/conflictivas de TensorFlow y dependencias
# ⚠️ Ejecuta este bloque ANTES del BLOQUE 4
# 💡 Esto evitará advertencias de compatibilidad en pip

import warnings
warnings.filterwarnings('ignore')

print("🧹 INICIANDO LIMPIEZA DE DEPENDENCIAS CONFLICTIVAS...\n")

# Paso 1: Limpiar caché de pip
print("📦 Limpiando caché de pip...")
!pip cache purge
print("   ✅ Caché de pip limpiado\n")

# Paso 2: Desinstalar versiones antiguas de TensorFlow
print("📦 Desinstalando versiones antiguas de TensorFlow...")
!pip uninstall -y -q tensorflow tensorflow-gpu tf-keras 2>/dev/null || true
print("   ✅ TensorFlow antiguo desinstalado\n")

# Paso 3: Desinstalar paquetes problemáticos que dependen de versiones específicas
print("📦 Desinstalando paquetes con dependencias rígidas...")
packages_to_remove = [
    "tensorflow-decision-forests",
    "dopamine-rl", 
    "tensorflow-text",
    "ydf"
]

for package in packages_to_remove:
    print(f"   - Desinstalando {package}...")
    !pip uninstall -y -q {package} 2>/dev/null || true

print("   ✅ Paquetes problemáticos desinstalados\n")

# Paso 4: Reinstalar TensorFlow 2.19 (versión de Colab)
print("📦 Reinstalando TensorFlow 2.19.0 (versión limpia)...")
!pip install -q --force-reinstall --no-cache-dir "tensorflow==2.19.0"
print("   ✅ TensorFlow 2.19.0 reinstalado limpiamente\n")

# Paso 5: Actualizar ml_dtypes y protobuf a versiones compatibles
print("📦 Actualizando ml_dtypes y protobuf...")
!pip install -q --upgrade --no-cache-dir "ml_dtypes>=0.5.0"
!pip install -q --upgrade --no-cache-dir "protobuf>=5.26.1,<6.0"
print("   ✅ ml_dtypes y protobuf actualizados\n")

# Paso 6: Verificar instalación limpia
print("🔍 Verificando instalación limpia...")
import tensorflow as tf
import numpy as np

try:
    import ml_dtypes
    ml_dtypes_version = ml_dtypes.__version__
except:
    ml_dtypes_version = "No disponible"

try:
    import google.protobuf
    protobuf_version = google.protobuf.__version__
except:
    protobuf_version = "No disponible"

print("\n✅ ESTADO DESPUÉS DE LA LIMPIEZA:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}")
print(f"   - ml_dtypes: {ml_dtypes_version}")
print(f"   - Protobuf: {protobuf_version}")

print("\n✅ LIMPIEZA COMPLETADA EXITOSAMENTE")
print("💡 Ahora ejecuta el BLOQUE 4 para instalar las dependencias sin conflictos")
print("💡 Si aún hay advertencias menores, puedes ignorarlas - no afectarán el funcionamiento")


In [ ]:
# ============================================================
# BLOQUE 4: INSTALACIÓN DE DEPENDENCIAS CRÍTICAS (VERSIÓN LIMPIA)
# ============================================================
# 🔧 Instala dependencias críticas después de la limpieza del BLOQUE 3.5
# ⚠️ Ejecuta SOLO después del BLOQUE 3.5
# ✅ Sin conflictos de versiones

import warnings
warnings.filterwarnings('ignore')

print("📦 INSTALANDO DEPENDENCIAS CRÍTICAS (ENTORNO LIMPIO)...\n")

# Verificar que TensorFlow está limpio
import tensorflow as tf
import numpy as np

print(f"🔍 Verificando versiones base:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}\n")

# Paso 1: Instalar MLflow con dependencias compatibles
print("📦 Instalando MLflow (compatible con NumPy 2.x y Protobuf 5.x)...")
!pip install -q --no-cache-dir "mlflow==2.16.2"
print("   ✅ MLflow instalado\n")

# Paso 2: Instalar DVC (opcional, para versionado de datos)
print("📦 Instalando DVC...")
!pip install -q --no-cache-dir "dvc[gs,s3]==3.51.1"
print("   ✅ DVC instalado\n")

# Paso 3: Actualizar scikit-learn
print("📦 Actualizando scikit-learn...")
!pip install -q --upgrade --no-cache-dir "scikit-learn>=1.6"
print("   ✅ Scikit-learn actualizado\n")

# Configurar mixed precision para GPU
print("🔍 Configurando TensorFlow...")
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("   ✅ Mixed precision (FP16) activado\n")

# Verificar versiones finales
print("=" * 60)
print("✅ DEPENDENCIAS CRÍTICAS INSTALADAS (ENTORNO LIMPIO):")
print("=" * 60)

print(f"\n📦 VERSIONES INSTALADAS:")
print(f"   - TensorFlow: {tf.__version__}")
print(f"   - NumPy: {np.__version__}")

try:
    import mlflow
    print(f"   - MLflow: {mlflow.__version__} ✅")
    mlflow_ok = True
except Exception as e:
    print(f"   - MLflow: Error - {e} ❌")
    mlflow_ok = False

try:
    import sklearn
    print(f"   - Scikit-learn: {sklearn.__version__}")
except:
    print(f"   - Scikit-learn: No disponible")

try:
    import google.protobuf
    print(f"   - Protobuf: {google.protobuf.__version__}")
except:
    print(f"   - Protobuf: No disponible")

try:
    import ml_dtypes
    print(f"   - ml_dtypes: {ml_dtypes.__version__}")
except:
    print(f"   - ml_dtypes: No disponible")

# Verificar compatibilidad
print(f"\n🔍 VERIFICACIÓN DE COMPATIBILIDAD:")
tf_version_ok = tf.__version__.startswith('2.19')
numpy_version_ok = np.__version__.startswith('2.0') or np.__version__.startswith('2.1')

print(f"   - TensorFlow 2.19.x: {'✅' if tf_version_ok else '⚠️'}")
print(f"   - NumPy 2.x: {'✅' if numpy_version_ok else '⚠️'}")
print(f"   - MLflow funcional: {'✅' if mlflow_ok else '❌'}")

if tf_version_ok and numpy_version_ok and mlflow_ok:
    print(f"\n" + "=" * 60)
    print("✅ INSTALACIÓN COMPLETADA EXITOSAMENTE (SIN CONFLICTOS)")
    print("=" * 60)
    print(f"\n💡 Continúa con el BLOQUE 5 para instalar complementos")
else:
    print(f"\n⚠️ Hay problemas de compatibilidad. Verifica los errores anteriores.")
    print(f"💡 Intenta reiniciar el runtime: Entorno de ejecución > Reiniciar sesión")

print(f"\n📝 NOTAS:")
print(f"   - Entorno limpio sin paquetes conflictivos")
print(f"   - TensorFlow 2.19 + NumPy 2.x + MLflow 2.16.2")
print(f"   - Mixed precision (FP16) configurado para GPU")
print(f"   - Sin advertencias de dependencias conflictivas")


In [ ]:
# ============================================================
# BLOQUE 5: INSTALACIÓN DE COMPLEMENTOS Y VERIFICACIONES FINALES
# ============================================================
# 🔧 Instala complementos: Albumentations, OpenCV, herramientas ML, etc.
# ⚠️ Requiere: BLOQUE 4 ejecutado exitosamente
# 💡 Este bloque instala herramientas adicionales y verifica todo el entorno

# ⚠️ Ejecuta este bloque DESPUÉS del BLOQUE 4. Instala herramientas adicionales
#    necesarias para el pipeline de datos y entrenamiento.

import warnings
warnings.filterwarnings('ignore')

# Paso 1: Instalar Albumentations y OpenCV (compatibles con numpy 1.26.4)
print("📦 Instalando Albumentations y OpenCV...")
!pip install -q "albumentations==2.0.8" "opencv-python-headless==4.10.0.84"

# Paso 2: Instalar herramientas de ML y datos
print("📦 Instalando herramientas de ML y datos...")
!pip install -q kaggle gdown plotly seaborn

# Paso 3: Instalar dependencias adicionales
print("📦 Instalando dependencias adicionales...")
!pip install -q "pillow>=11.0.0" "packaging>=24.0" google-images-download==2.8.0

# Verificar versiones instaladas
print("\n🔍 Verificando complementos instalados...")
import numpy as np
import cv2
import albumentations as A
import sklearn

print("\n✅ COMPLEMENTOS INSTALADOS:")
print(f"   - NumPy: {np.__version__}")
print(f"   - OpenCV: {cv2.__version__}")
print(f"   - Albumentations: {A.__version__}")
print(f"   - Scikit-learn: {sklearn.__version__}")

# Verificar TensorFlow y GPU (configuración final)
import tensorflow as tf
from tensorflow.keras import mixed_precision

# Verificar versión de TensorFlow instalada
print(f"\n✅ TensorFlow: {tf.__version__}")

# Configurar GPU si está disponible
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU detectada y configurada correctamente")
        print(f"   Dispositivos GPU: {len(gpus)}")
        for i, gpu in enumerate(gpus):
            print(f"   - GPU {i}: {gpu.name}")
        print("\n🎮 GPU lista para entrenamiento.")
    except RuntimeError as e:
        print(f"⚠️ Error configurando GPU: {e}")
else:
    print("\n⚠️ No se detectó GPU.")
    print("💡 Activa GPU desde: Entorno de ejecución > Cambiar tipo de entorno > Acelerador de hardware > GPU")
    print("💡 Sin GPU, el entrenamiento será más lento pero funcionará correctamente.")

# Verificar que mixed precision está activado
print(f"\n✅ Mixed precision (FP16) activado para acelerar entrenamiento en GPU")

print(f"\n✅ TODAS LAS DEPENDENCIAS INSTALADAS CORRECTAMENTE")
print(f"💡 Puedes continuar con el BLOQUE 6 (Imports Generales)")
print(f"💡 Las advertencias de pip sobre otros paquetes (shap, jax, etc.) no afectan el funcionamiento.")


In [ ]:
# ============================================================
# BLOQUE 6: IMPORTS Y CONFIGURACIÓN GENERAL
# ============================================================
# 🔍 Importa todas las librerías necesarias (pandas, numpy, tensorflow, mlflow, etc.)

# 🔍 Conjunto completo de librerías usadas en el pipeline: utilidades del sistema,
#    ciencia de datos, visualización, ML y tracking de experimentos.
import os
import sys
import shutil
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import json
import requests
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# MLflow para tracking reproducible de experimentos.
import mlflow
import mlflow.tensorflow

# Albumentations y OpenCV (requieren BLOQUE 5 ejecutado)
try:
    import cv2
    import albumentations as A
    cv2_version = cv2.__version__
    albumentations_version = A.__version__
    cv2_available = True
except ImportError as e:
    cv2_version = "No disponible (ejecuta BLOQUE 5)"
    albumentations_version = "No disponible (ejecuta BLOQUE 5)"
    cv2_available = False

# Configurar matplotlib para que todas las gráficas se vean consistentes en Colab.
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Todas las dependencias importadas correctamente")
print(f"📊 Versiones:")
print(f"   - TensorFlow: {tf.__version__}")
if cv2_available:
    print(f"   - OpenCV: {cv2_version}")
    print(f"   - Albumentations: {albumentations_version}")
else:
    print(f"   - OpenCV: {cv2_version}")
    print(f"   - Albumentations: {albumentations_version}")


In [ ]:
# ============================================================
# BLOQUE 7: CONFIGURACIÓN DEL PROYECTO Y ESTRUCTURA DE CARPETAS
# ============================================================
# ⚙️ Crea estructura de carpetas para datos y modelos en Drive
# 📁 Usa el mismo BASE_DIR del BLOQUE 1 (proyecto ya clonado en Drive)
# 💡 Drive ya está montado en el BLOQUE 1, así que solo verificamos

from pathlib import Path
import mlflow

# 🔗 Verificar que Drive está montado (ya debería estar montado en el BLOQUE 1)
drive_path = Path('/content/drive')
if not drive_path.exists() or not any(drive_path.iterdir()):
    print("⚠️ Google Drive no está montado. Montando ahora...")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive montado exitosamente")
    except Exception as e:
        print(f"❌ Error al montar Google Drive: {e}")
        raise RuntimeError("Google Drive debe estar montado. Ejecuta el BLOQUE 1 primero.")
else:
    print('✅ Google Drive ya está montado (montado en el BLOQUE 1)')

# 📁 Directorio base dentro de tu Drive (mismo que el BLOQUE 1)
#    El proyecto ya está clonado aquí desde el BLOQUE 1
BASE_DIR = Path('/content/drive/MyDrive/bovine-weight-estimation')

# Verificar que el proyecto existe (debería existir desde el BLOQUE 1)
if not BASE_DIR.exists():
    print(f"⚠️ El proyecto no existe en {BASE_DIR}")
    print("💡 Ejecuta el BLOQUE 1 primero para clonar el repositorio en Drive")
    raise RuntimeError(f"El proyecto debe existir en {BASE_DIR}. Ejecuta el BLOQUE 1 primero.")
else:
    print(f"✅ Proyecto encontrado en: {BASE_DIR}")

# 📂 Creamos (si no existen) las carpetas estándar para datos crudos, procesados y modelos.
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
AUGMENTED_DIR = DATA_DIR / 'augmented'
MODELS_DIR = BASE_DIR / 'models'
MLRUNS_DIR = BASE_DIR / 'mlruns'

for dir_path in [DATA_DIR, RAW_DIR, PROCESSED_DIR, AUGMENTED_DIR, MODELS_DIR, MLRUNS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# 📊 Configuración de MLflow (tracking local persistente)
# ------------------------------------------------------------
mlflow.set_tracking_uri(f"file://{MLRUNS_DIR}")
mlflow.set_experiment("bovine-weight-estimation")

# ------------------------------------------------------------
# ⚙️ Configuración general del entrenamiento (hiperparámetros base)
# ------------------------------------------------------------
CONFIG = {
    'image_size': (224, 224),
    'batch_size': 32,
    'epochs': 100,
    'learning_rate': 0.001,
    'validation_split': 0.2,
    'test_split': 0.1,
    'early_stopping_patience': 10,
    'target_r2': 0.95,
    'max_mae': 5.0,
    'max_inference_time': 3.0
}

# ------------------------------------------------------------
# 🐄 Razas objetivo (Santa Cruz, Chiquitanía y Pampa)
# ------------------------------------------------------------
BREEDS = [
    'brahman', 'nelore', 'angus', 'cebuinas',
    'criollo', 'pardo_suizo', 'guzerat', 'holstein'
]

print("✅ Configuración completada correctamente")
print(f"📁 Directorio base: {BASE_DIR}")
print(f"🎯 Razas objetivo: {len(BREEDS)} razas -> {BREEDS}")
print(f"📊 MLflow tracking: {MLRUNS_DIR}")


## 📥 Día 2-3: Descargar y Organizar Datasets Críticos


In [ ]:
# ============================================================
# BLOQUE 7.5: CONFIGURAR VARIABLES DE ENTORNO PARA DATASETS
# ============================================================
# 📁 Configura rutas del CID Dataset y metadata antes de ejecutar BLOQUE 8
# ⚠️ Ejecuta ANTES del BLOQUE 8 si tienes el CID Dataset en Drive
# 👉 Ajusta las rutas según donde estén tus archivos en Google Drive

import os
from pathlib import Path

# 👉 Ruta del archivo comprimido del CID Dataset (zip, tar.gz, etc.)
#    Ajusta según donde subiste el archivo en Drive
CID_DATASET_ARCHIVE_PATH = '/content/drive/MyDrive/bovine-weight-estimation/data/raw/cid_dataset.zip'
# CID_DATASET_ARCHIVE_PATH = '/content/drive/MyDrive/datasets/cid_dataset.tar.gz'  # Ejemplo alternativo

# 👉 Ruta del archivo metadata.csv del CID Dataset
#    Ajusta según donde esté tu metadata.csv en Drive
CID_METADATA_FILE = '/content/drive/MyDrive/bovine-weight-estimation/data/raw/cid/metadata.csv'
# CID_METADATA_FILE = '/content/drive/MyDrive/datasets/cid_metadata.csv'  # Ejemplo alternativo

# Configurar variables de entorno
os.environ['CID_DATASET_ARCHIVE_PATH'] = CID_DATASET_ARCHIVE_PATH
os.environ['CID_METADATA_FILE'] = CID_METADATA_FILE

# Verificar que los archivos existan
archive_path = Path(CID_DATASET_ARCHIVE_PATH)
metadata_path = Path(CID_METADATA_FILE)

if archive_path.exists():
    print(f"✅ Archivo comprimido CID encontrado: {archive_path}")
    print(f"   Tamaño: {archive_path.stat().st_size / 1024 / 1024:.2f} MB")
else:
    print(f"⚠️ Archivo comprimido CID NO encontrado en: {archive_path}")
    print("💡 Sube el archivo comprimido del CID Dataset a Drive antes de ejecutar BLOQUE 8")

if metadata_path.exists():
    print(f"✅ Metadata CSV encontrado: {metadata_path}")
else:
    print(f"⚠️ Metadata CSV NO encontrado en: {metadata_path}")
    print("💡 Sube el archivo metadata.csv del CID Dataset a Drive antes de ejecutar BLOQUE 12")

print("\n📋 Variables de entorno configuradas:")
print(f"   CID_DATASET_ARCHIVE_PATH = {CID_DATASET_ARCHIVE_PATH}")
print(f"   CID_METADATA_FILE = {CID_METADATA_FILE}")



In [ ]:
# ============================================================
# BLOQUE 8.5: CONFIGURAR KAGGLE.JSON DESDE DRIVE (OPCIONAL)
# ============================================================
# 🔑 Copia kaggle.json desde Google Drive a /root/.kaggle/
# ⚠️ Ejecuta SOLO si tienes kaggle.json en Drive y no lo has configurado aún
# 📁 Ajusta la ruta KAGGLE_JSON_PATH según donde esté tu archivo en Drive

from pathlib import Path
import shutil
import subprocess

# 👉 Ajusta esta ruta a donde esté tu kaggle.json en Google Drive
KAGGLE_JSON_PATH = Path('/content/drive/MyDrive/keys/kaggle.json')  # <--- CAMBIA ESTA RUTA
# KAGGLE_JSON_PATH = Path('/content/drive/MyDrive/bovine-weight-estimation/kaggle.json')  # Ejemplo alternativo

if KAGGLE_JSON_PATH.exists():
    kaggle_dir = Path('/root/.kaggle')
    kaggle_dir.mkdir(exist_ok=True)
    
    shutil.copy(KAGGLE_JSON_PATH, kaggle_dir / 'kaggle.json')
    subprocess.run(["chmod", "600", "/root/.kaggle/kaggle.json"], check=True)
    
    print("✅ kaggle.json copiado desde Drive a /root/.kaggle/")
    print("🔑 Credenciales de Kaggle configuradas correctamente")
else:
    print(f"⚠️ No se encontró kaggle.json en: {KAGGLE_JSON_PATH}")
    print("💡 Ajusta KAGGLE_JSON_PATH o sube kaggle.json manualmente antes del BLOQUE 9")



In [ ]:
# ============================================================
# BLOQUE 8: CID DATASET (17,899 imágenes)
# ============================================================
# 📥 Extrae el CID Dataset desde archivo comprimido subido a Drive
# ⚠️ Requiere: Variable CID_DATASET_ARCHIVE_PATH apuntando al archivo .zip/.tar.gz

CID_DATASET_ARCHIVE_PATH = os.environ.get('CID_DATASET_ARCHIVE_PATH')


def download_cid_dataset(archive_path: str | None = CID_DATASET_ARCHIVE_PATH) -> Path:
    """Prepara el CID Dataset desde un archivo previamente descargado.

    Requisitos antes de ejecutar:
    1. Sube el archivo comprimido real (zip/tar) al directorio definido en BASE_DIR o /content.
    2. Establece la variable de entorno CID_DATASET_ARCHIVE_PATH apuntando a ese archivo.

    No se generan datos sintéticos: si falta el archivo, se detendrá con un error.
    """
    cid_dir = RAW_DIR / 'cid'
    cid_dir.mkdir(parents=True, exist_ok=True)

    if any(cid_dir.iterdir()):
        print(f"ℹ️ CID Dataset ya está disponible en {cid_dir}. Se omite extracción.")
        return cid_dir

    if archive_path is None:
        raise RuntimeError(
            "Configura la variable de entorno CID_DATASET_ARCHIVE_PATH con la ruta del "
            "archivo comprimido del CID Dataset (por ejemplo .zip o .tar.gz) antes de ejecutar esta celda."
        )

    archive_path = Path(archive_path)
    if not archive_path.exists():
        raise FileNotFoundError(
            f"No se encontró el archivo comprimido del CID Dataset en {archive_path}. "
            "Sube el dataset real a tu Google Drive y vuelve a ejecutar."
        )

    print(f"📥 Extrayendo CID Dataset desde: {archive_path}")
    try:
        shutil.unpack_archive(str(archive_path), str(cid_dir))
    except shutil.ReadError as exc:
        raise RuntimeError(
            "No se pudo desempaquetar el CID Dataset. Verifica que el archivo esté en un formato soportado "
            "(.zip, .tar, .tar.gz, .tar.bz2, etc.)."
        ) from exc

    if not any(cid_dir.iterdir()):
        raise RuntimeError(
            "La extracción del CID Dataset no produjo archivos. Verifica que el archivo comprimido contenga datos válidos."
        )

    print(f"✅ CID Dataset preparado en: {cid_dir}")
    return cid_dir


# Ejecutar preparación (requerirá archivo real previamente cargado)
cid_dataset_path = download_cid_dataset()


In [ ]:
# ============================================================
# BLOQUE 9: KAGGLE CATTLE WEIGHT DATASET (12k imágenes)
# ============================================================
# 📥 Descarga dataset de Kaggle usando API
# ⚠️ Requiere: kaggle.json subido a /root/.kaggle/ (ver instrucciones)

KAGGLE_DATASET_ID = os.environ.get(
    'KAGGLE_DATASET_ID', 'sadhliroomyprime/cattle-weight-detection-model-dataset-12k'
)


def setup_kaggle_api() -> Path:
    """Configura la API de Kaggle para descargas reales."""
    print("🔑 Configurando API de Kaggle...")

    kaggle_dir = Path('/root/.kaggle')
    kaggle_dir.mkdir(exist_ok=True)

    kaggle_json = kaggle_dir / 'kaggle.json'
    if not kaggle_json.exists():
        raise FileNotFoundError(
            "No se encontró /root/.kaggle/kaggle.json. Descarga tu token desde "
            "https://www.kaggle.com/account, súbelo al notebook y vuelve a ejecutar."
        )

    subprocess.run(["chmod", "600", "/root/.kaggle/kaggle.json"], check=True)
    return kaggle_dir


def download_kaggle_dataset(dataset_id: str = KAGGLE_DATASET_ID) -> Path:
    """Descarga el dataset de Kaggle indicado.

    Requisitos:
    - Subir `kaggle.json` (token API) a este notebook y colocarlo en /root/.kaggle/
    - Definir KAGGLE_DATASET_ID si deseas descargar un dataset distinto al preset.
    """
    if not dataset_id:
        raise RuntimeError("Define la variable de entorno KAGGLE_DATASET_ID con el dataset a descargar.")

    kaggle_dir = setup_kaggle_api()
    output_dir = RAW_DIR / 'kaggle'
    output_dir.mkdir(parents=True, exist_ok=True)

    if any(output_dir.glob('**/*')):
        print(f"ℹ️ Dataset de Kaggle ya presente en {output_dir}. Se omite descarga.")
        return output_dir

    print(f"📥 Descargando dataset de Kaggle: {dataset_id}")
    subprocess.run([
        "kaggle",
        "datasets",
        "download",
        "-d",
        dataset_id,
        "-p",
        str(output_dir),
    ], check=True)

    archive_files = list(output_dir.glob('*.zip'))
    if not archive_files:
        raise RuntimeError("La descarga de Kaggle no produjo archivos .zip. Verifica el ID del dataset.")

    for archive_file in archive_files:
        print(f"📦 Descomprimiendo {archive_file.name}")
        subprocess.run([
            "unzip",
            "-q",
            str(archive_file),
            "-d",
            str(output_dir),
        ], check=True)
        archive_file.unlink()

    if not any(output_dir.glob('**/*')):
        raise RuntimeError("La extracción del dataset de Kaggle no produjo archivos. Revisa el contenido descargado.")

    print(f"✅ Kaggle dataset disponible en: {output_dir}")
    return output_dir


# Ejecutar descarga (requiere credenciales reales)
kaggle_dataset_path = download_kaggle_dataset()


In [ ]:
# ============================================================
# BLOQUE 10: GOOGLE IMAGES SCRAPING (OPCIONAL)
# ============================================================
# 🖼️ Descarga imágenes de Google Images para razas locales
# ⚠️ Opcional: Solo ejecuta si necesitas complementar datasets
# ⚠️ Cuidado: Respeta términos de uso y evita bloqueos

def scrape_google_images():
    """Scraping de Google Images para razas locales.

    Uso opcional para complementar razas poco representadas. Respeta los términos de uso
    del motor de búsqueda y evita ejecutar múltiples veces para no ser bloqueado.
    """
    print("🖼️ Scraping Google Images para razas locales...")
    
    from google_images_download import google_images_download
    
    # Razas locales específicas
    breeds_local = [
        'ganado criollo boliviano',
        'guzerat bolivia', 
        'brahman chiquitania',
        'nelore pantanal',
        'angus bolivia',
        'pardo suizo bolivia',
        'jersey bolivia'
    ]
    
    response = google_images_download.googleimagesdownload()
    
    scraped_count = 0
    
    for breed in breeds_local:
        try:
            print(f"📸 Scraping: {breed}")
            
            # Configuración de descarga
            arguments = {
                "keywords": breed,
                "limit": 50,  # Límite por término
                "print_urls": False,
                "output_directory": str(RAW_DIR / 'scraped'),
                "image_directory": breed.replace(' ', '_'),
                "format": "jpg",
                "size": "medium",
                "aspect_ratio": "wide"
            }
            
            # Descargar imágenes
            paths = response.download(arguments)
            
            if paths:
                count = len(paths[0])
                scraped_count += count
                print(f"✅ {breed}: {count} imágenes descargadas")
            
        except Exception as e:
            print(f"⚠️ Error con {breed}: {e}")
            continue
    
    print(f"🎯 Total imágenes scraped: {scraped_count}")
    return scraped_count

# Ejecutar scraping
scraped_images = scrape_google_images()


In [ ]:
# ============================================================
# BLOQUE 11: RESUMEN DE DATASETS DESCARGADOS
# ============================================================
# 📊 Muestra resumen de todos los datasets disponibles
# ⚠️ Requiere: BLOQUE 8, 9, 10 ejecutados (o al menos uno)

def summarize_datasets(cid_df: pd.DataFrame | None = None) -> pd.DataFrame:
    """Resumen de todos los datasets disponibles (solo datos reales)."""
    print("📊 RESUMEN DE DATASETS")
    print("=" * 50)

    datasets_info = []

    if cid_df is not None:
        datasets_info.append({
            'name': 'CID Dataset',
            'images': len(cid_df),
            'description': 'Computer Vision Research - Cattle Image Database',
            'status': '✅ Disponible',
        })
    else:
        datasets_info.append({
            'name': 'CID Dataset',
            'images': 0,
            'description': 'CID sin metadata cargada',
            'status': '⚠️ Pendiente',
        })

    # Kaggle Dataset (maneja caso cuando aún no existe)
    try:
        kaggle_path = kaggle_dataset_path if 'kaggle_dataset_path' in globals() else None
        kaggle_id = KAGGLE_DATASET_ID if 'KAGGLE_DATASET_ID' in globals() else 'N/A'
        if kaggle_path and Path(kaggle_path).exists():
            kaggle_images = len(list(Path(kaggle_path).glob('**/*.jpg')))
            datasets_info.append({
                'name': 'Kaggle Cattle Weight',
                'images': kaggle_images,
                'description': f'Dataset Kaggle ({kaggle_id})',
                'status': '✅ Disponible' if kaggle_images > 0 else '⚠️ Vacío',
            })
        else:
            datasets_info.append({
                'name': 'Kaggle Cattle Weight',
                'images': 0,
                'description': 'Requiere configuración de API Kaggle',
                'status': '⚠️ Pendiente',
            })
    except NameError:
        datasets_info.append({
            'name': 'Kaggle Cattle Weight',
            'images': 0,
            'description': 'Requiere ejecutar BLOQUE 9',
            'status': '⚠️ Pendiente',
        })

    # Google Images Scraped (maneja caso cuando aún no existe)
    try:
        scraped_count = scraped_images if 'scraped_images' in globals() else 0
        datasets_info.append({
            'name': 'Google Images Scraped',
            'images': scraped_count,
            'description': 'Razas locales bolivianas',
            'status': '✅ Disponible' if scraped_count > 0 else '⚠️ Pendiente',
        })
    except NameError:
        datasets_info.append({
            'name': 'Google Images Scraped',
            'images': 0,
            'description': 'Requiere ejecutar BLOQUE 10 (opcional)',
            'status': '⚠️ Pendiente',
        })

    df_datasets = pd.DataFrame(datasets_info)
    print(df_datasets.to_string(index=False))

    total_images = int(df_datasets['images'].sum())
    print(f"\n🎯 TOTAL IMÁGENES DISPONIBLES: {total_images:,}")

    summary_path = DATA_DIR / 'datasets_summary.csv'
    df_datasets.to_csv(summary_path, index=False)
    print(f"\n💾 Resumen guardado en: {summary_path}")

    return df_datasets

# Ejecutar resumen (maneja caso cuando df_cid aún no existe)
# ⚠️ Nota: Si ejecutas ANTES del BLOQUE 12, df_cid será None y solo mostrará datasets de Kaggle/Google
try:
    # Verificar si df_cid existe en el scope global
    df_cid_temp = df_cid if 'df_cid' in globals() else None
    datasets_summary = summarize_datasets(df_cid_temp)
except NameError:
    # Si df_cid no existe aún, ejecutar resumen sin metadata del CID
    print("ℹ️ df_cid aún no cargado. Ejecuta BLOQUE 12 para cargar metadata del CID Dataset.")
    datasets_summary = summarize_datasets(None)


## 📊 Día 4: Análisis Exploratorio de Datos (EDA)


In [ ]:
# ============================================================
# BLOQUE 12: ANÁLISIS EXPLORATORIO - CID DATASET
# ============================================================
# 📊 Carga y analiza metadata del CID Dataset
# ⚠️ Requiere: CID_METADATA_FILE apuntando a metadata.csv

CID_METADATA_FILE = Path(os.environ.get('CID_METADATA_FILE', cid_dataset_path / 'metadata.csv'))


def analyze_cid_dataset(metadata_file: Path) -> pd.DataFrame:
    """Análisis exploratorio utilizando datos reales del CID Dataset."""
    if not metadata_file.exists():
        raise FileNotFoundError(
            f"No se encontró el archivo de metadata del CID Dataset en {metadata_file}. "
            "Genera o coloca un CSV con las columnas ['image_path', 'weight_kg', 'breed', 'age_category', 'image_quality', 'lighting', 'angle']."
        )

    df_cid = pd.read_csv(metadata_file)

    required_columns = {
        'image_path',
        'weight_kg',
        'breed',
        'age_category',
        'image_quality',
        'lighting',
        'angle',
    }
    missing_columns = required_columns.difference(df_cid.columns)
    if missing_columns:
        raise ValueError(
            f"La metadata del CID Dataset no contiene las columnas requeridas: {sorted(missing_columns)}"
        )

    print("📊 ANÁLISIS EXPLORATORIO - CID DATASET")
    print("=" * 60)
    print(f"📈 Total imágenes: {len(df_cid):,}")
    print(f"📊 Dimensiones: {df_cid.shape}")

    print("\n📋 Columnas disponibles:")
    for col in df_cid.columns:
        print(f"  - {col}")

    print("\n⚖️ DISTRIBUCIÓN DE PESO:")
    print(df_cid['weight_kg'].describe())

    print("\n🐄 DISTRIBUCIÓN POR RAZA:")
    print(df_cid['breed'].value_counts())

    print("\n📸 CALIDAD DE IMÁGENES:")
    print(df_cid['image_quality'].value_counts())

    return df_cid


# Ejecutar análisis (requiere metadata real)
df_cid = analyze_cid_dataset(CID_METADATA_FILE)


In [ ]:
# ============================================================
# BLOQUE 13: VISUALIZACIONES EDA
# ============================================================
# 📊 Crea gráficos interactivos del análisis exploratorio
# ⚠️ Requiere: BLOQUE 12 ejecutado (df_cid cargado)

def create_eda_visualizations(df):
    """Crear visualizaciones completas del EDA"""
    print("📊 Creando visualizaciones EDA...")
    
    # Configurar subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Distribución de Peso', 'Peso por Raza',
            'Distribución por Edad', 'Calidad de Imágenes',
            'Peso vs Iluminación', 'Peso vs Ángulo'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Distribución de peso
    fig.add_trace(
        go.Histogram(x=df['weight_kg'], nbinsx=50, name='Peso (kg)',
                    marker_color='lightblue', opacity=0.7),
        row=1, col=1
    )
    
    # 2. Peso por raza
    for breed in df['breed'].unique():
        breed_data = df[df['breed'] == breed]['weight_kg']
        fig.add_trace(
            go.Box(y=breed_data, name=breed, boxpoints='outliers'),
            row=1, col=2
        )
    
    # 3. Distribución por edad
    age_counts = df['age_category'].value_counts()
    fig.add_trace(
        go.Bar(x=age_counts.index, y=age_counts.values, name='Categorías de Edad',
               marker_color='lightgreen'),
        row=2, col=1
    )
    
    # 4. Calidad de imágenes
    quality_counts = df['image_quality'].value_counts()
    fig.add_trace(
        go.Pie(labels=quality_counts.index, values=quality_counts.values,
               name='Calidad'),
        row=2, col=2
    )
    
    # 5. Peso vs Iluminación
    for lighting in df['lighting'].unique():
        lighting_data = df[df['lighting'] == lighting]['weight_kg']
        fig.add_trace(
            go.Box(y=lighting_data, name=lighting),
            row=3, col=1
        )
    
    # 6. Peso vs Ángulo
    for angle in df['angle'].unique():
        angle_data = df[df['angle'] == angle]['weight_kg']
        fig.add_trace(
            go.Box(y=angle_data, name=angle),
            row=3, col=2
        )
    
    # Configurar layout
    fig.update_layout(
        height=1200,
        title_text="Análisis Exploratorio - CID Dataset",
        title_x=0.5,
        showlegend=True
    )
    
    # Mostrar gráfico
    fig.show()
    
    # Guardar gráfico
    fig.write_html(DATA_DIR / 'eda_visualizations.html')
    print(f"💾 Visualizaciones guardadas en: {DATA_DIR / 'eda_visualizations.html'}")
    
    return fig

# Ejecutar visualizaciones
eda_fig = create_eda_visualizations(df_cid)


In [ ]:
# ============================================================
# BLOQUE 14: ANÁLISIS ESPECÍFICO POR RAZA
# ============================================================
# 🐄 Analiza qué razas tienen suficientes datos para entrenamiento
# ⚠️ Requiere: BLOQUE 12 ejecutado (df_cid cargado)

def analyze_breeds_for_training(df):
    """Analizar qué razas están bien representadas para entrenamiento"""
    print("🐄 ANÁLISIS POR RAZA PARA ENTRENAMIENTO")
    print("=" * 50)
    
    # Razas objetivo del proyecto
    target_breeds = ['brahman', 'nelore', 'angus', 'cebuinas', 'criollo', 'pardo_suizo', 'jersey']
    
    breed_analysis = []
    
    for breed in target_breeds:
        # Buscar razas similares en el dataset
        if breed in df['breed'].values:
            breed_data = df[df['breed'] == breed]
            count = len(breed_data)
            avg_weight = breed_data['weight_kg'].mean()
            std_weight = breed_data['weight_kg'].std()
            
            status = "✅ Suficiente" if count >= 1000 else "⚠️ Limitado" if count >= 100 else "❌ Insuficiente"
            
        else:
            # Buscar razas similares
            similar_breeds = []
            if breed in ['brahman', 'nelore', 'cebuinas']:
                similar_breeds = ['mixed']  # Bos indicus
            elif breed in ['angus']:
                similar_breeds = ['mixed']  # Bos taurus
            
            count = sum(len(df[df['breed'] == sb]) for sb in similar_breeds)
            avg_weight = df[df['breed'].isin(similar_breeds)]['weight_kg'].mean() if similar_breeds else 0
            std_weight = df[df['breed'].isin(similar_breeds)]['weight_kg'].std() if similar_breeds else 0
            
            status = "🔄 Transfer Learning" if count >= 1000 else "❌ Recolección requerida"
        
        breed_analysis.append({
            'breed': breed,
            'images_available': count,
            'avg_weight_kg': round(avg_weight, 1),
            'std_weight_kg': round(std_weight, 1),
            'status': status,
            'strategy': 'Direct training' if count >= 1000 else 'Transfer learning' if count >= 100 else 'Data collection'
        })
    
    # Crear DataFrame
    df_breed_analysis = pd.DataFrame(breed_analysis)
    
    # Mostrar tabla
    print(df_breed_analysis.to_string(index=False))
    
    # Guardar análisis
    df_breed_analysis.to_csv(DATA_DIR / 'breed_analysis.csv', index=False)
    print(f"\n💾 Análisis por raza guardado en: {DATA_DIR / 'breed_analysis.csv'}")
    
    # Recomendaciones
    print(f"\n🎯 RECOMENDACIONES:")
    
    sufficient_breeds = df_breed_analysis[df_breed_analysis['images_available'] >= 1000]
    if len(sufficient_breeds) > 0:
        print(f"✅ Entrenamiento directo: {', '.join(sufficient_breeds['breed'].tolist())}")
    
    transfer_breeds = df_breed_analysis[(df_breed_analysis['images_available'] >= 100) & (df_breed_analysis['images_available'] < 1000)]
    if len(transfer_breeds) > 0:
        print(f"🔄 Transfer learning: {', '.join(transfer_breeds['breed'].tolist())}")
    
    collection_breeds = df_breed_analysis[df_breed_analysis['images_available'] < 100]
    if len(collection_breeds) > 0:
        print(f"📸 Recolección requerida: {', '.join(collection_breeds['breed'].tolist())}")
    
    return df_breed_analysis

# Ejecutar análisis por raza
breed_analysis = analyze_breeds_for_training(df_cid)


## 🔧 Día 5-6: Preparar Pipeline de Datos


In [ ]:
# ============================================================
# BLOQUE 15: PIPELINE DE DATOS OPTIMIZADO
# ============================================================
# 🔧 Crea pipeline de datos con augmentation para entrenamiento
# ⚠️ Requiere: BLOQUE 12 ejecutado (df_cid cargado)

class CattleDataPipeline:
    """Pipeline de datos para entrenamiento de modelos de estimación de peso"""
    
    def __init__(self, data_dir, breeds_mapping=None):
        self.data_dir = Path(data_dir)
        self.breeds_mapping = breeds_mapping or {}
        
        # Augmentation agresivo para datasets pequeños
        self.augmentation = A.Compose([
            # Variaciones de iluminación
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.6),
            A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, p=0.5),
            
            # Ruido y desenfoque
            A.GaussNoise(var_limit=(5, 15), p=0.3),
            A.Blur(blur_limit=3, p=0.25),
            
            # Efectos atmosféricos
            A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), p=0.4),
            A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=0.2),
            
            # Transformaciones geométricas
            A.RandomRotate90(p=0.3),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(
                shift_limit=0.1, scale_limit=0.15, 
                rotate_limit=15, border_mode=cv2.BORDER_REFLECT, p=0.5
            ),
            
            # Augmentation específico para ganado
            A.RandomCrop(height=200, width=200, p=0.3),  # Simular diferentes distancias
            A.ElasticTransform(alpha=1, sigma=50, p=0.2),  # Deformaciones naturales
            A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.2),
        ])
        
        print(f"✅ Pipeline inicializado para: {self.data_dir}")
    
    def load_and_preprocess(self, img_path: Path, weight: float) -> tuple[np.ndarray, float]:
        """Carga imagen, aplica augmentation y retorna tensores listos para el modelo."""
        if not img_path.exists():
            raise FileNotFoundError(f"Imagen no encontrada: {img_path}")

        img = cv2.imread(str(img_path))
        if img is None:
            raise ValueError(f"No se pudo cargar la imagen: {img_path}")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        augmented = self.augmentation(image=img)
        img = augmented['image']

        img = cv2.resize(img, CONFIG['image_size'])
        img = img.astype(np.float32) / 255.0

        return img, float(weight)

    def create_tf_dataset(self, df, split='train'):
        """Crea un tf.data.Dataset a partir de rutas reales."""
        print(f"🔧 Creando dataset TensorFlow para split: {split}")

        required_columns = {'image_path', 'weight_kg'}
        missing_columns = required_columns.difference(df.columns)
        if missing_columns:
            raise ValueError(
                f"El DataFrame para el split '{split}' no contiene las columnas requeridas: {sorted(missing_columns)}"
            )

        def data_generator():
            for _, row in df.iterrows():
                raw_path = Path(row['image_path'])
                img_path = raw_path if raw_path.is_absolute() else self.data_dir / raw_path

                img, weight = self.load_and_preprocess(img_path, row['weight_kg'])
                yield img, weight

        dataset = tf.data.Dataset.from_generator(
            data_generator,
            output_signature=(
                tf.TensorSpec(shape=CONFIG['image_size'] + (3,), dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32),
            ),
        )

        dataset = dataset.cache()

        if split == 'train':
            dataset = dataset.shuffle(1000)

        dataset = dataset.batch(CONFIG['batch_size'])
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        print(f"✅ Dataset {split} creado con optimizaciones")
        return dataset
    
    def split_data(self, df):
        """Divide datos en train/val/test"""
        print("📊 Dividiendo datos en train/val/test...")
        
        # Shuffle datos
        df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Calcular splits
        n_total = len(df_shuffled)
        n_train = int(n_total * (1 - CONFIG['validation_split'] - CONFIG['test_split']))
        n_val = int(n_total * CONFIG['validation_split'])
        
        # Dividir
        df_train = df_shuffled[:n_train]
        df_val = df_shuffled[n_train:n_train + n_val]
        df_test = df_shuffled[n_train + n_val:]
        
        print(f"📈 Train: {len(df_train):,} ({len(df_train)/n_total*100:.1f}%)")
        print(f"📈 Val: {len(df_val):,} ({len(df_val)/n_total*100:.1f}%)")
        print(f"📈 Test: {len(df_test):,} ({len(df_test)/n_total*100:.1f}%)")
        
        return df_train, df_val, df_test

# Crear pipeline
pipeline = CattleDataPipeline(RAW_DIR)

# Dividir datos
df_train, df_val, df_test = pipeline.split_data(df_cid)

# Crear datasets TensorFlow
train_dataset = pipeline.create_tf_dataset(df_train, 'train')
val_dataset = pipeline.create_tf_dataset(df_val, 'val')
test_dataset = pipeline.create_tf_dataset(df_test, 'test')


In [ ]:
# ============================================================
# BLOQUE 16: ARQUITECTURA DEL MODELO
# ============================================================
# 🏗️ Crea modelo EfficientNetB0 con transfer learning
# ⚠️ Requiere: BLOQUE 15 ejecutado (pipeline creado)

def create_weight_estimation_model():
    """Crear modelo para estimación de peso"""
    print("🏗️ Creando arquitectura del modelo...")
    
    # Base model con transfer learning
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=CONFIG['image_size'] + (3,)
    )
    
    # Congelar capas iniciales
    base_model.trainable = False
    
    # Custom head para regresión
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu', name='dense_1')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu', name='dense_2')(x)
    x = layers.Dropout(0.2)(x)
    
    # Salida: peso estimado en kg
    output = layers.Dense(1, activation='linear', name='weight_output')(x)
    
    # Crear modelo
    model = models.Model(inputs=base_model.input, outputs=output)
    
    # Compilar modelo
    model.compile(
        optimizer=optimizers.Adam(learning_rate=CONFIG['learning_rate']),
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    print(f"✅ Modelo creado con {model.count_params():,} parámetros")
    print(f"📊 Arquitectura: EfficientNetB0 + Custom Head")
    
    return model

# Crear modelo
model = create_weight_estimation_model()

# Mostrar resumen
model.summary()


In [ ]:
# ============================================================
# BLOQUE 17: CONFIGURACIÓN DE ENTRENAMIENTO
# ============================================================
# ⚙️ Configura callbacks (EarlyStopping, ReduceLR, ModelCheckpoint, TensorBoard)
# ⚠️ Requiere: BLOQUE 16 ejecutado (modelo creado)

def setup_training_callbacks():
    """Configurar callbacks para entrenamiento"""
    print("⚙️ Configurando callbacks de entrenamiento...")
    
    callbacks_list = [
        # Early stopping
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['early_stopping_patience'],
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce learning rate on plateau
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Model checkpoint
        callbacks.ModelCheckpoint(
            filepath=str(MODELS_DIR / 'best_model.h5'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        
        # TensorBoard
        callbacks.TensorBoard(
            log_dir=str(BASE_DIR / 'logs'),
            histogram_freq=1,
            write_graph=True,
            write_images=True
        )
    ]
    
    print(f"✅ {len(callbacks_list)} callbacks configurados")
    return callbacks_list

# Configurar callbacks
training_callbacks = setup_training_callbacks()

# Configurar MLflow
def start_mlflow_run():
    """Iniciar run de MLflow"""
    run = mlflow.start_run(run_name="cattle-weight-base-model")

    mlflow.log_params({
        'dataset': 'CID',
        'model': 'EfficientNetB0',
        'batch_size': CONFIG['batch_size'],
        'learning_rate': CONFIG['learning_rate'],
        'epochs': CONFIG['epochs'],
        'image_size': CONFIG['image_size'],
        'augmentation': 'Albumentations'
    })

    print(f"🔬 MLflow run iniciado: {run.info.run_id}")
    return run

# Iniciar MLflow run
mlflow_run = start_mlflow_run()


In [ ]:
# ============================================================
# BLOQUE 18: ENTRENAMIENTO DEL MODELO
# ============================================================
# 🚀 Entrena el modelo base (puede tardar horas con GPU)
# ⚠️ Requiere: BLOQUE 17 ejecutado (callbacks configurados)
# ⚠️ Tiempo estimado: 2-4 horas con GPU T4 (100 épocas)

def train_model():
    """Entrenar modelo base"""
    print("🚀 Iniciando entrenamiento del modelo base...")
    print(f"📊 Configuración: {CONFIG}")
    
    # Calcular steps por época
    steps_per_epoch = len(df_train) // CONFIG['batch_size']
    validation_steps = len(df_val) // CONFIG['batch_size']
    
    print(f"📈 Steps por época: {steps_per_epoch}")
    print(f"📈 Validation steps: {validation_steps}")
    
    # Entrenar modelo
    history = model.fit(
        train_dataset,
        epochs=CONFIG['epochs'],
        validation_data=val_dataset,
        callbacks=training_callbacks,
        verbose=1
    )
    
    print("✅ Entrenamiento completado")
    return history

# Entrenamiento real (requiere datasets preparados y tiempo de ejecución con GPU)
history = train_model()


In [ ]:
# ============================================================
# BLOQUE 19: EVALUACIÓN DEL MODELO
# ============================================================
# 📊 Evalúa el modelo en conjunto de test (calcula R², MAE, MSE)
# ⚠️ Requiere: BLOQUE 18 ejecutado (modelo entrenado)

def evaluate_model():
    """Evaluar modelo en conjunto de test"""
    print("📊 Evaluando modelo en conjunto de test...")
    
    # Evaluar modelo
    test_loss, test_mae, test_mse = model.evaluate(test_dataset, verbose=0)

    # Calcular R² real con predicciones sobre el conjunto de test
    y_true = []
    y_pred = []
    for batch_images, batch_targets in test_dataset:
        predictions = model.predict(batch_images, verbose=0)
        y_true.extend(batch_targets.numpy().astype(float))
        y_pred.extend(predictions.squeeze().astype(float))

    test_r2 = r2_score(y_true, y_pred)

    print(f"📈 RESULTADOS DE EVALUACIÓN:")
    print(f"   Loss: {test_loss:.2f}")
    print(f"   MAE: {test_mae:.2f} kg")
    print(f"   MSE: {test_mse:.2f}")
    print(f"   R²: {test_r2:.3f}")
    
    # Verificar objetivos
    print(f"\n🎯 VERIFICACIÓN DE OBJETIVOS:")
    print(f"   R² ≥ {CONFIG['target_r2']}: {'✅' if test_r2 >= CONFIG['target_r2'] else '❌'} ({test_r2:.3f})")
    print(f"   MAE < {CONFIG['max_mae']} kg: {'✅' if test_mae < CONFIG['max_mae'] else '❌'} ({test_mae:.2f} kg)")
    
    # Log métricas en MLflow
    mlflow.log_metrics({
        'test_loss': test_loss,
        'test_mae': test_mae,
        'test_mse': test_mse,
        'test_r2': test_r2
    })
    
    return {
        'loss': test_loss,
        'mae': test_mae,
        'mse': test_mse,
        'r2': test_r2
    }

# Evaluar modelo
evaluation_results = evaluate_model()


In [ ]:
# ============================================================
# BLOQUE 20: EXPORTAR A TFLITE
# ============================================================
# 📱 Exporta modelo entrenado a TFLite optimizado para móvil
# ⚠️ Requiere: BLOQUE 19 ejecutado (modelo evaluado)
# 📁 Guarda modelo en MODELS_DIR/generic-cattle-v1.0.0.tflite

def export_to_tflite(model, output_path):
    """Exporta modelo a TFLite optimizado para móvil"""
    print(f"📱 Exportando modelo a TFLite: {output_path}")
    
    # Configurar conversor
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # Optimizaciones para móvil
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]  # FP16 para velocidad
    
    # Cuantización INT8 (opcional, más agresiva)
    # converter.representative_dataset = representative_data_gen
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    
    # Convertir
    tflite_model = converter.convert()
    
    # Guardar
    with open(output_path, 'wb') as f:
        f.write(tflite_model)
    
    # Información del modelo
    model_size_kb = len(tflite_model) / 1024
    model_size_mb = model_size_kb / 1024
    print("✅ Modelo exportado exitosamente")
    print(f"📏 Tamaño: {model_size_mb:.2f} MB ({model_size_kb:.1f} KB)")
    print("📱 Optimizado para móvil: FP16")
    
    # Log en MLflow
    mlflow.log_artifact(output_path)
    mlflow.log_metric('model_size_kb', model_size_kb)
    mlflow.log_metric('model_size_mb', model_size_mb)
    
    return model_size_kb

# Exportar modelo base
tflite_path = MODELS_DIR / 'generic-cattle-v1.0.0.tflite'
model_size = export_to_tflite(model, tflite_path)

print("\n🎯 MODELO BASE LISTO PARA INTEGRACIÓN")
print(f"📁 Archivo: {tflite_path}")
print(f"📏 Tamaño: {model_size / 1024:.2f} MB ({model_size:.1f} KB)")
print(f"🔬 MLflow run: {mlflow_run.info.run_id}")


## 📋 Resumen y Próximos Pasos


In [ ]:
# ============================================================
# BLOQUE 21: RESUMEN FINAL
# ============================================================
# 📋 Genera resumen completo del trabajo realizado
# ⚠️ Requiere: Todos los bloques anteriores ejecutados
# 💾 Guarda resumen en DATA_DIR/final_summary.json

def generate_final_summary():
    """Generar resumen final del trabajo realizado"""
    print("📋 RESUMEN FINAL - PERSONA 2: SETUP ML")
    print("=" * 60)
    
    # Resumen de datasets
    print(f"\n📥 DATASETS PROCESADOS:")
    cid_row = datasets_summary[datasets_summary['name'] == 'CID Dataset']
    cid_images = int(cid_row['images'].iloc[0]) if not cid_row.empty else 0
    print(f"   {'✅' if cid_images else '⚠️'} CID Dataset: {cid_images:,} imágenes")
    print(f"   {'✅' if scraped_images else '⚠️'} Google Images: {scraped_images:,} imágenes locales")

    if kaggle_dataset_path and kaggle_dataset_path.exists():
        kaggle_images = len(list(kaggle_dataset_path.glob('**/*.jpg')))
        status_icon = '✅' if kaggle_images else '⚠️'
        print(f"   {status_icon} Kaggle Dataset ({KAGGLE_DATASET_ID}): {kaggle_images:,} imágenes")
    else:
        print("   ⚠️ Kaggle Dataset: Pendiente configuración (sube kaggle.json y ejecuta la celda correspondiente)")
    
    # Resumen de análisis
    print(f"\n📊 ANÁLISIS COMPLETADO:")
    print(f"   ✅ EDA completo con visualizaciones")
    print(f"   ✅ Análisis por raza para estrategia de entrenamiento")
    print(f"   ✅ Pipeline de datos optimizado")
    
    # Resumen de modelo
    print(f"\n🤖 MODELO BASE:")
    print(f"   ✅ Arquitectura: EfficientNetB0 + Custom Head")
    print(f"   ✅ Parámetros: {model.count_params():,}")
    print(f"   ✅ TFLite exportado: {model_size / 1024:.2f} MB ({model_size:.1f} KB)")
    print(f"   ✅ MLflow tracking: {mlflow_run.info.run_id}")
    
    # Próximos pasos
    print(f"\n🎯 PRÓXIMOS PASOS:")
    print(f"   1. 🔄 Fine-tuning por raza (Semanas 3-6)")
    print(f"   2. 📸 Recolección Criollo + Pardo Suizo (Semanas 7-8)")
    print(f"   3. 🧪 Entrenamiento final (Semanas 9-10)")
    print(f"   4. 📱 Integración en app móvil")
    
    # Guardar resumen
    summary_data = {
        'completion_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'datasets_processed': len(datasets_summary),
        'total_images': datasets_summary['images'].sum(),
        'model_architecture': 'EfficientNetB0',
        'model_size_kb': model_size,
        'mlflow_run_id': mlflow_run.info.run_id,
        'status': 'COMPLETADO'
    }
    
    with open(DATA_DIR / 'final_summary.json', 'w') as f:
        json.dump(summary_data, f, indent=2)

    mlflow.end_run()
    
    print(f"\n💾 Resumen guardado en: {DATA_DIR / 'final_summary.json'}")
    print(f"\n🎉 PERSONA 2: SETUP ML COMPLETADO EXITOSAMENTE")

# Generar resumen final
generate_final_summary()


## 📝 Notas Importantes

### ⚠️ Configuración Requerida
1. **Kaggle API**: Subir `kaggle.json` para descargar datasets
2. **CID Dataset**: Reemplazar URL simulada con URL real
3. **CattleEyeView**: Solicitar acceso a autores del paper

### 🔧 Optimizaciones Implementadas
- **Mixed Precision**: FP16 para acelerar entrenamiento
- **Data Pipeline**: Cache + prefetch + shuffle optimizado
- **Augmentation**: Albumentations específico para ganado
- **TFLite Export**: Optimizado para móvil

### 📊 Métricas Objetivo
- **R² ≥ 0.95**: Explicación 95% de varianza
- **MAE < 5 kg**: Error absoluto promedio
- **Inference < 3s**: Tiempo en móvil

### 🎯 Estado Actual
- ✅ **Infraestructura ML**: Completada
- ✅ **Pipeline de datos**: Optimizado
- ✅ **Modelo base**: Listo para fine-tuning
- 🔄 **Próximo**: Fine-tuning por raza específica
